# Workflow

In [ ]:
import grandpy as gp

sars = gp.read_grand("https://zenodo.org/record/5834034/files/sars.tsv.gz", design=("Condition", "dur.4sU", "Replicate"), classify_genes_func=lambda df: gp.classify_genes(df, cg_name="viral"))

In [ ]:
sars.coldata

In [ ]:
sars.gene_info

In [ ]:
sars = sars.filter_genes(min_expression=1000)
len(sars.genes)

In [ ]:
sars = sars.normalize()

In [ ]:
sars.get_table()

In [ ]:
gp.plot_pca(sars, aest={"color": "duration.4sU.original", "shape": "Condition"})

In [ ]:
gp.plot_gene_progressive_timecourse(sars, "SMAD3", steady_state={"Mock": True, "SARS": False})

In [ ]:
sars = sars.calibrate_effective_labeling_time_kinetic_fit(steady_state={"Mock": True, "SARS": False}, n_top_genes=10)

In [ ]:
gp.plot_gene_progressive_timecourse(sars, "SMAD3", time="calibrated_time", steady_state={"Mock": True, "SARS": False}, exact_tics=False)

In [ ]:
sars = sars.fit_kinetics(time="calibrated_time", steady_state={"Mock": True, "SARS": False})

In [ ]:
sars.get_analyses(description=True)

In [ ]:
gp.plot_scatter(sars, x = "kinetics_SARS_Half-life", y = "kinetics_Mock_Half-life", diagonal=True, limit=(0,25) ,analysis=True)

In [ ]:
sars = sars.compute_lfc(contrasts = sars.get_contrasts())
sars.get_analysis_table("total")